# Natural Language Processing (NLP) with Named Entity Recognition (NER)

 Install Java, PySpark, and Spark NLP


In [ ]:
import os
# Install java
! apt-get update -qq
! apt-get install -y openjdk-8-jdk-headless -qq > /dev/null
os.environ["JAVA_HOME"] = "/usr/lib/jvm/java-8-openjdk-amd64"
os.environ["PATH"] = os.environ["JAVA_HOME"] + "/bin:" + os.environ["PATH"]
! java -version
# Install pyspark
#! pip install --ignore-installed pyspark==3.1.2
# Install Spark NLP
#! pip install --ignore-installed spark-nlp==3.4.3

W: Skipping acquire of configured file 'main/source/Sources' as repository 'https://r2u.stat.illinois.edu/ubuntu jammy InRelease' does not seem to provide it (sources.list entry misspelt?)
openjdk version "1.8.0_422"
OpenJDK Runtime Environment (build 1.8.0_422-8u422-b05-1~22.04-b05)
OpenJDK 64-Bit Server VM (build 25.422-b05, mixed mode)
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 6.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 17.3 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880746 sha256=6f43d4f4fe4113a4f4119aba4c16143e4905742fb3ed40d8fd4156c680f505f8
  Stored in directory: /root/.cache/pip/wheels/ef/70/50/7882e1bcb5693225f7cc86698f10953201b48b3f36317c2d18
Successfully built pyspark
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 4.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.6/144.6 kB 11.3 MB/s eta 0:00:00


In [ ]:
# Uninstall any pre-existing pyspark installations
!pip uninstall -y pyspark
!pip uninstall -y spark-nlp

# Install the exact versions you need
!pip install pyspark==3.1.2
!pip install spark-nlp==3.4.3




Found existing installation: pyspark 3.5.3
Uninstalling pyspark-3.5.3:
  Successfully uninstalled pyspark-3.5.3
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 212.4/212.4 MB 5.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 198.6/198.6 kB 13.5 MB/s eta 0:00:00
  Created wheel for pyspark: filename=pyspark-3.1.2-py2.py3-none-any.whl size=212880746 sha256=70e8b721d90cb5732eb8bbadb173bb461448eccd79c5045bee95272e32966de3
  Stored in directory: /root/.cache/pip/wheels/ef/70/50/7882e1bcb5693225f7cc86698f10953201b48b3f36317c2d18
Successfully built pyspark
  Attempting uninstall: py4j
    Found existing installation: py4j 0.10.9.7
    Uninstalling py4j-0.10.9.7:
      Successfully uninstalled py4j-0.10.9.7
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 53.0/53.0 kB 3.4 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 144.6/144.6 kB 11.1 MB/s eta 0:00:00


TypeError: start() got an unexpected keyword argument 'spark31'

**Ready to use PySpark and Spark NLP.**

##Import the relevant packages

In [ ]:
# Start Spark Session
from pyspark.sql import SparkSession
from pyspark.sql import SparkSession
from pyspark.ml import Pipeline
from sparknlp.annotator import *
from sparknlp.common import *
from sparknlp.base import *
import sparknlp

spark = sparknlp.start()
print("Spark NLP version: ", sparknlp.version())
print("Apache Spark version: ", spark.version)

Spark NLP version:  3.4.3
Apache Spark version:  3.1.2


**The Spark NLP is 3.4.3 and Apache Spark versions  just installed is 3.1.2**

**Download the CoNLL dataset**

In [ ]:
from pathlib import Path
import urllib.request
download_path = "./eng.train"
if not Path(download_path).is_file():
 print("File Not found will downloading it!")
 url = "https://github.com/patverga/torch-ner-nlp-from-scratch/raw/master/data/conll2003/eng.train"
 urllib.request.urlretrieve(url, download_path)
else:
 print("File already present.")

File Not found will downloading it!


##2.2.4 Convert the CoNLL file to Spark data frame

Then we convert the CoNLL file to Spark data frame with all the additional fields generated to
be used later on.

In [ ]:
from sparknlp.training import CoNLL
training_data = CoNLL().readDataset(spark, './eng.train')
training_data.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|                text|            document|            sentence|               token|                 pos|               label|
+--------------------+--------------------+--------------------+--------------------+--------------------+--------------------+
|EU rejects German...|[{document, 0, 47...|[{document, 0, 47...|[{token, 0, 1, EU...|[{pos, 0, 1, NNP,...|[{named_entity, 0...|
|     Peter Blackburn|[{document, 0, 14...|[{document, 0, 14...|[{token, 0, 4, Pe...|[{pos, 0, 4, NNP,...|[{named_entity, 0...|
| BRUSSELS 1996-08-22|[{document, 0, 18...|[{document, 0, 18...|[{token, 0, 7, BR...|[{pos, 0, 7, NNP,...|[{named_entity, 0...|
|The European Comm...|[{document, 0, 18...|[{document, 0, 18...|[{token, 0, 2, Th...|[{pos, 0, 2, DT, ...|[{named_entity, 0...|
|Germany 's repres...|[{document, 0, 21...|[{document, 0, 21...|[{token, 0, 6, Ge...|[{pos, 0, 6, NNP,..

##2.2.5 Word embedding
The next step is to get the word embeddings through BERT. We will use a Spark NLP annotator
called BertEmbeddings().

In [ ]:
from sparknlp.annotator import BertEmbeddings
bert = BertEmbeddings.pretrained('bert_base_cased', 'en') \
.setInputCols(["sentence",'token'])\
.setOutputCol("bert")\
.setCaseSensitive(False)

bert_base_cased download started this may take some time.
Approximate size to download 389.1 MB
[OK!]


In Spark NLP, we have four pre-trained variants of BERT

• bert_base_uncased

• bert_base_cased

• bert_large_uncased

• bert_large_cased

‘Which one should be used?’ depends on your use case, training set, and the complexity of the
task you are trying to model.
In the code snippet above, we basically load the bert_base_cased version from Spark NLP public
resources and point the sentence and token columns in setInputCols(). In short,
BertEmbeddings() annotator will take the sentence and token columns and populate Bert
embeddings in the bert column. In general, each word is translated to a 768-dimensional vector.

##2.2.6 Configuring Tagger

Then we import the NerDLApproach() annotator, the main module responsible for training the
NER model.

In [ ]:
from sparknlp.annotator import NerDLApproach # Import NerDLApproach
nerTagger = NerDLApproach()\
.setInputCols(["sentence", "token", "bert"])\
.setLabelColumn("label")\
.setOutputCol("ner")\
.setMaxEpochs(1)\
.setRandomSeed(0)\
.setVerbose(1)\
.setValidationSplit(0.2)\
.setEvaluationLogExtended(True)\
.setEnableOutputLogs(True)\
.setIncludeConfidence(True)\
.setTestDataset("test_withEmbeds.parquet")

.setInputCols([“sentence”, “token”, “bert”]) : the columns that will be used by the NER
model to generate features.
• .setLabelColumn(“label”) : the target columns
• .setOutputCol(“ner”) : the predictions will be written to the ner column
• .setMaxEpochs(1) : number of epochs for training
• .setVerbose(1) : the level of logs while training
• .setValidationSplit(0.2): the proportion of the training dataset to be validated against the
model on each Epoch. The value should be between 0.0 and 1.0 and by default, it is 0.0
and off.
• .setEvaluationLogExtended(True): Whether logs for validation to be extended: it displays
the time and evaluation of each label. The default is false.
• .setEnableOutputLogs(True): Whether to output to the log folder. When set to True, the
logs and training metrics will be written to the folder in the home folder.
• .setIncludeConfidence(True): whether to include confidence scores in annotation
metadata.
• .setTestDataset(“test_withEmbeds.parquet”): The path to test dataset. If set, it is used to
calculate statistics on it during training. This is also in a CoNLL format, but embeddings
are added through and saved to disk as before. You don’t have to set this if you don’t
need to evaluate your model on an unseen test set in Spark.

##2.2.7.1 Download testing data


In [ ]:
from pathlib import Path
import urllib.request
download_path = "./eng.testa"
if not Path(download_path).is_file():
 print("File Not found will downloading it!")
 url = "https://github.com/patverga/torch-ner-nlp-from-scratch/raw/master/data/conll2003/eng.testa"
 urllib.request.urlretrieve(url, download_path)
else:
 print("File already present.")

File Not found will downloading it!


##2.2.7.2 Transform and save
Transform and save it as a parquet file. Apache Parquet is a columnar storage format

In [ ]:
test_data = CoNLL().readDataset(spark, './eng.testa')
test_data = bert.transform(test_data)
test_data.write.mode("overwrite").parquet("test_withEmbeds.parquet")

We  can also set the learning rate (setLr), learning rate decay coefficient (setPo), setBatchSize,
and setDropout rate.

Finally, the test result will be saved into test_withEmbeds.parquet file


##2.2.8 Training NER Model
Now we can append these two annotators in a pipeline, then train the NER model with the
training dataset

In [ ]:
from pyspark.ml import Pipeline

In [ ]:

%%time
ner_pipeline = Pipeline(stages = [bert, nerTagger])
ner_model = ner_pipeline.fit(training_data.limit(500))

CPU times: user 1.22 s, sys: 140 ms, total: 1.36 s
Wall time: 3min 31s


##2.2.9 Testing (prediction)

In [ ]:
predictions = ner_model.transform(test_data.select("sentence","token","label"))
predictions.show()

+--------------------+--------------------+--------------------+--------------------+--------------------+
|            sentence|               token|               label|                bert|                 ner|
+--------------------+--------------------+--------------------+--------------------+--------------------+
|[{document, 0, 64...|[{token, 0, 6, CR...|[{named_entity, 0...|[{word_embeddings...|[{named_entity, 0...|
|[{document, 0, 16...|[{token, 0, 5, LO...|[{named_entity, 0...|[{word_embeddings...|[{named_entity, 0...|
|[{document, 0, 18...|[{token, 0, 3, We...|[{named_entity, 0...|[{word_embeddings...|[{named_entity, 0...|
|[{document, 0, 20...|[{token, 0, 4, Th...|[{named_entity, 0...|[{word_embeddings...|[{named_entity, 0...|
|[{document, 0, 21...|[{token, 0, 4, Af...|[{named_entity, 0...|[{word_embeddings...|[{named_entity, 0...|
|[{document, 0, 12...|[{token, 0, 7, Tr...|[{named_entity, 0...|[{word_embeddings...|[{named_entity, 0...|
|[{document, 0, 16...|[{token, 0, 4, 

In [ ]:
predictions.select("token.result","label.result","ner.result").show(truncate=40)

+----------------------------------------+----------------------------------------+----------------------------------------+
|                                  result|                                  result|                                  result|
+----------------------------------------+----------------------------------------+----------------------------------------+
|[CRICKET, -, LEICESTERSHIRE, TAKE, OV...|   [O, O, I-ORG, O, O, O, O, O, O, O, O]|   [O, O, I-PER, O, O, O, O, O, O, O, O]|
|                    [LONDON, 1996-08-30]|                              [I-LOC, O]|                              [I-LOC, O]|
|[West, Indian, all-rounder, Phil, Sim...|[I-MISC, I-MISC, O, I-PER, I-PER, O, ...|[O, I-PER, O, I-PER, I-PER, O, O, O, ...|
|[Their, stay, on, top, ,, though, ,, ...|[O, O, O, O, O, O, O, O, O, O, O, O, ...|[O, O, O, O, O, O, O, O, O, O, O, O, ...|
|[After, bowling, Somerset, out, for, ...|[O, O, I-ORG, O, O, O, O, O, O, O, O,...|[O, O, I-PER, O, O, O, O, O, O, O, O,...|


In [ ]:
predictions.printSchema()

root
 |-- sentence: array (nullable = false)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (nullable = true)
 |    |    |    |-- element: float (containsNull = false)
 |-- token: array (nullable = false)
 |    |-- element: struct (containsNull = true)
 |    |    |-- annotatorType: string (nullable = true)
 |    |    |-- begin: integer (nullable = false)
 |    |    |-- end: integer (nullable = false)
 |    |    |-- result: string (nullable = true)
 |    |    |-- metadata: map (nullable = true)
 |    |    |    |-- key: string
 |    |    |    |-- value: string (valueContainsNull = true)
 |    |    |-- embeddings: array (n

In [ ]:
import pyspark.sql.functions as F
predictions.select(F.explode(F.arrays_zip('token.result','label.result','ner.result')).alias("cols")) \
.select(F.expr("cols['0']").alias("token"),
        F.expr("cols['1']").alias("ground_truth"),
        F.expr("cols['2']").alias("prediction")).show(truncate=False)

+--------------+------------+----------+
|token         |ground_truth|prediction|
+--------------+------------+----------+
|CRICKET       |O           |O         |
|-             |O           |O         |
|LEICESTERSHIRE|I-ORG       |I-PER     |
|TAKE          |O           |O         |
|OVER          |O           |O         |
|AT            |O           |O         |
|TOP           |O           |O         |
|AFTER         |O           |O         |
|INNINGS       |O           |O         |
|VICTORY       |O           |O         |
|.             |O           |O         |
|LONDON        |I-LOC       |I-LOC     |
|1996-08-30    |O           |O         |
|West          |I-MISC      |O         |
|Indian        |I-MISC      |I-PER     |
|all-rounder   |O           |O         |
|Phil          |I-PER       |I-PER     |
|Simmons       |I-PER       |I-PER     |
|took          |O           |O         |
|four          |O           |O         |
+--------------+------------+----------+
only showing top


**Annotate (NER) a text using a PretrainedPipeline (recognize_entities_dl) in SparkNLP [10][11]**
**and show the results of annotated (named entity) text**
**• Input Text from Wikipedia**

The University of Illinois Springfield (UIS) is a public university in Springfield,
Illinois, United States. As a public liberal arts college and the newest campus in
the University of Illinois system, UIS is a member of the Council of Public Liberal
Arts Colleges. UIS is also part of the American Association of State Colleges and
Universities and the American Council on Education. The campus' main repository,
Brookens Library, holds a collection of nearly 800,000 books and serials in addition
to accessible resources at the University of Illinois Chicago and University of
Illinois Urbana-Champaign campuses. President: Timothy L. Killeen. Chancellor: Janet
L. Gooch. Location: Springfield, Illinois, United States.

**To show the named entity of each word, the output style should be like the one below.**
[('The', 'O'),
 ('University', 'B-ORG'),
 ('of', 'I-ORG'),
 ('Illinois', 'I-ORG'),
 ('Springfield', 'I-ORG'),
 ('(', 'O'),
 ('UIS', 'B-ORG'),
 (')', 'O'),
 ('is', 'O'),
 ('a', 'O'),
 ('public', 'O'),
... (skipped)

In [ ]:
from sparknlp.pretrained import PretrainedPipeline
pretrain_pipeline = PretrainedPipeline('recognize_entities_dl', lang='en')
nert="The University of Illinois Springfield (UIS) is a public university in Springfield, Illinois, United States. As a public liberal arts college and the newest campus in the University of Illinois system, UIS is a member of the Council of Public Liberal Arts Colleges. UIS is also part of the American Association of State Colleges and Universities and the American Council on Education. The campus' main repository, Brookens Library, holds a collection of nearly 800,000 books and serials in addition to accessible resources at the University of Illinois Chicago and University of Illinois Urbana-Champaign campuses. President: Timothy L. Killeen. Chancellor: Janet L. Gooch. Location: Springfield, Illinois, United States."
res=pretrain_pipeline.annotate(nert)
list(zip(res['token'], res['ner']))

recognize_entities_dl download started this may take some time.
Approx size to download 160.1 MB
[OK!]


[('The', 'O'),
 ('University', 'B-ORG'),
 ('of', 'I-ORG'),
 ('Illinois', 'I-ORG'),
 ('Springfield', 'I-ORG'),
 ('(', 'O'),
 ('UIS', 'B-ORG'),
 (')', 'O'),
 ('is', 'O'),
 ('a', 'O'),
 ('public', 'O'),
 ('university', 'O'),
 ('in', 'O'),
 ('Springfield', 'B-LOC'),
 (',', 'O'),
 ('Illinois', 'B-LOC'),
 (',', 'O'),
 ('United', 'B-LOC'),
 ('States', 'I-LOC'),
 ('.', 'O'),
 ('As', 'O'),
 ('a', 'O'),
 ('public', 'O'),
 ('liberal', 'O'),
 ('arts', 'O'),
 ('college', 'O'),
 ('and', 'O'),
 ('the', 'O'),
 ('newest', 'O'),
 ('campus', 'O'),
 ('in', 'O'),
 ('the', 'O'),
 ('University', 'B-ORG'),
 ('of', 'I-ORG'),
 ('Illinois', 'I-ORG'),
 ('system', 'O'),
 (',', 'O'),
 ('UIS', 'B-ORG'),
 ('is', 'O'),
 ('a', 'O'),
 ('member', 'O'),
 ('of', 'O'),
 ('the', 'O'),
 ('Council', 'B-ORG'),
 ('of', 'I-ORG'),
 ('Public', 'I-ORG'),
 ('Liberal', 'I-ORG'),
 ('Arts', 'I-ORG'),
 ('Colleges', 'I-ORG'),
 ('.', 'O'),
 ('UIS', 'B-ORG'),
 ('is', 'O'),
 ('also', 'O'),
 ('part', 'O'),
 ('of', 'O'),
 ('the', 'O'),
 ('Amer